# Implementación Entorno

In [ ]:
%pip install google-colab-selenium bs4

In [43]:
# Librerías Necesarias
import google_colab_selenium as gs
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

import time

# Conexión y Extracción a El Peruano

In [31]:
# Definición de Periodo de Extracción
fecha_inicio = "12/09/2025"
fecha_final = "12/09/2025"

In [39]:
# Iniciacilización de la Conexión
driver = gs.Chrome()
driver.get("https://diariooficial.elperuano.pe/Normas")
time.sleep(20) # Tiempo Promedio para que cargue la página

# Modificación de Periodo de Extracción
wait = WebDriverWait(driver, 20)

input_desde = wait.until(EC.presence_of_element_located((By.ID, "cddesde")))
input_hasta = wait.until(EC.presence_of_element_located((By.ID, "cdhasta")))

driver.execute_script("arguments[0].removeAttribute('readonly'); arguments[0].removeAttribute('contenteditable');", input_desde)
driver.execute_script("arguments[0].removeAttribute('readonly'); arguments[0].removeAttribute('contenteditable');", input_hasta)

## Limpieza de Input y Colocación del nuevo período
input_desde.clear(); input_hasta.clear()
input_desde.send_keys(fecha_inicio); input_hasta.send_keys(fecha_final)

## Inicia la Búsqueda
boton_buscar = wait.until(EC.element_to_be_clickable((By.ID, "btnBuscar")))
boton_buscar.click()
time.sleep(25) # Tiempo Promedio para que cargue la nueva página

# Extracción del HTML de la página
html = driver.page_source

driver.quit()

<IPython.core.display.Javascript object>

# Procesamiento de la extracción

In [40]:
# Obtención de todos las normas (<article>)
soup = BeautifulSoup(html, "lxml")
articles = soup.find_all("article")

print("Cantidad de artículos encontrados:", len(articles))

Cantidad de artículos encontrados: 71


In [46]:
# Procesamiento y Extracción de los campos requeridos
data = []
for articulo in articles:

  # Título de la Norma (h4)
  h4 = articulo.find("h4").get_text(strip=True)

  # Nombre de la Norma (h5)
  h5 = articulo.find("h5")
  h5_oficial = h5.get_text(strip=True)

  # URL de la Norma (h5 a)
  a = h5.find("a")["href"]

  # Fecha de Publicación de la Norma (p)
  fecha = articulo.find("p")
  fecha = fecha.find("b").get_text(strip=True)
  fecha = fecha.replace("Fecha: ", "")

  # Descripción de la Norma (p)
  descripcion = articulo.find_all("p")[-1].get_text(strip=True)

  data.append({'titulo': h4, 'Nombre': h5_oficial, 'Enlace': a, 'Fecha': fecha, "Descripcion": descripcion})

# Presentación de Resultados

In [47]:
df = pd.DataFrame(data)
df.head()

,titulo,Nombre,Enlace,Fecha,Descripcion
0,PRESIDENCIA DEL CONSEJO DE MINISTROS,RESOLUCION SUPREMA N° 192-2025-PCM,https://busquedas.elperuano.pe/dispositivo/NL/...,12/09/2025,Autorizan viaje del Ministro de Transportes y ...
1,CULTURA,RESOLUCION DIRECTORAL N° 001738-2025-DE-DDC-C...,https://busquedas.elperuano.pe/dispositivo/NL/...,12/09/2025,Aprueban el Manual de Clasificador de Cargos d...
2,INTERIOR,RESOLUCION MINISTERIAL N° 1775-2025-IN,https://busquedas.elperuano.pe/dispositivo/NL/...,12/09/2025,Autorizan viaje de personal policial a Italia ...
3,JUSTICIA Y DERECHOS HUMANOS,RESOLUCION SUPREMA N° 184-2025-JUS,https://busquedas.elperuano.pe/dispositivo/NL/...,12/09/2025,Acceden a solicitud de extradición activa de c...
4,JUSTICIA Y DERECHOS HUMANOS,RESOLUCION SUPREMA N° 185-2025-JUS,https://busquedas.elperuano.pe/dispositivo/NL/...,12/09/2025,Acceden a solicitud de extradición activa de c...
